 # How to use the BRON arangodb
 
 You can view the public version of BRON with the arangodb web interface at [http:bron.alfa.csail.mit.edu:8529] with `username` `guest` and `password` `guest`.

In [1]:
# Change working directory
import os

cwd = os.path.split(os.getcwd())
if cwd[-1] == "tutorials":
    os.chdir("..")

assert os.path.split(os.getcwd())[-1] == "bron_framework"

In [2]:
from offense.build_software_and_groups import build_software, OUT_DATA_DIR
from utils.mitigation_utils import query_bron_aql

save_path = os.path.join("tests", OUT_DATA_DIR)

import arango

BRON_SERVER_IP = "localhost"
BRON_USERNAME = "guest"
BRON_PASSWORD = "guest"
DB = "BRON"

In [5]:
build_software(save_path, BRON_USERNAME, BRON_PASSWORD, BRON_SERVER_IP)

TypeError: query_bron_aql() missing 1 required positional argument: 'db'

In [3]:
client = arango.ArangoClient(hosts=f"http://{BRON_SERVER_IP}:8529")
db = client.db(
    "BRON", username=BRON_USERNAME, password=BRON_PASSWORD, auth_method="basic"
)
technique_bron = db.collection("technique")
print(technique_bron, technique_bron.name)

<StandardCollection technique> technique


In [4]:
print(save_path)
original_id = "T1001"
query_bron_aql(technique_bron.name, original_id, db)

tests/data/attacks


{'_key': 'technique_00015',
 '_id': 'technique/technique_00015',
 '_rev': '_emJuify---',
 'original_id': 'T1001',
 'datatype': 'technique',
 'name': 'Data Obfuscation',
 'metadata': {'description': 'Adversaries may obfuscate command and control traffic to make it more difficult to detect. Command and control (C2) communications are hidden (but not necessarily encrypted) in an attempt to make the content more difficult to discover or decipher and to make the communication less conspicuous and hide commands from being seen. This encompasses many methods, such as adding junk data to protocol traffic, using steganography, or impersonating legitimate protocols. ',
  'short_description': 'Adversaries may obfuscate command and control traffic to make it more difficult to detect. Command and control (C2) communications are hidden (but not necessarily encrypted) in an attempt to make the content more difficult to discover or decipher and to make the communication less conspicuous and hide comma

## Example connect to BRON arangodb and perform AQL query

In [5]:
%%time
query = """
FOR c IN tactic
    FILTER c.original_id == "TA0006"
    RETURN c
"""
client = arango.ArangoClient(hosts=f"http://{BRON_SERVER_IP}:8529")
db = client.db(DB, username=BRON_USERNAME, password=BRON_PASSWORD, auth_method="basic")
assert db.aql.validate(query)
cursor = db.aql.execute(query)
for c in cursor:
    print(c)

{'_key': 'tactic_00003', '_id': 'tactic/tactic_00003', '_rev': '_emJukVy--A', 'original_id': 'TA0006', 'datatype': 'tactic', 'name': 'credential-access', 'metadata': {'description': 'The adversary is trying to steal account names and passwords.\n\nCredential Access consists of techniques for stealing credentials like account names and passwords. Techniques used to get credentials include keylogging or credential dumping. Using legitimate credentials can give adversaries access to systems, make them harder to detect, and provide the opportunity to create more accounts to help achieve their goals.', 'short_description': 'The adversary is trying to steal account names and passwords.'}}
CPU times: user 4.65 ms, sys: 658 µs, total: 5.31 ms
Wall time: 5.3 ms


In [4]:
%%time
query_template = """
FOR c IN capec
    FILTER c.original_id == "{}"
    FOR v IN 1..1 ANY c {}
        RETURN v
"""
capec = "117"
edge_collections = ("TechniqueCapec", "CapecCwe")
for edge_collection in edge_collections:
    query = query_template.format(capec, edge_collection)
    assert db.aql.validate(query)
    print(query)
    cursor = db.aql.execute(query)
    results = {_["original_id"] for _ in cursor}
    print(f"{len(results)} result for CAPEC {capec} in {edge_collection}: {results}")


FOR c IN capec
    FILTER c.original_id == "117"
    FOR v IN 1..1 ANY c TechniqueCapec
        RETURN v

1 result for CAPEC 117 in TechniqueCapec: {'T1020.001'}

FOR c IN capec
    FILTER c.original_id == "117"
    FOR v IN 1..1 ANY c CapecCwe
        RETURN v

1 result for CAPEC 117 in CapecCwe: {'319'}
CPU times: user 11.7 ms, sys: 2.93 ms, total: 14.6 ms
Wall time: 172 ms


## Example report download, threat data extraction and BRON query

- Given a URL (in this example https://media.defense.gov/2020/Dec/17/2002554125/-1/-1/0/AUTHENTICATION_MECHANISMS_CSA_U_OO_198854_20.PDF )
- Parse the document at the URL (HTML or PDF) and use regexp to identify Tactics, Techniques and CVEs
- Query the collections in BRON with the extracted information 
- Prints
 - the queried records for each collection and the number of edges from each record
 - the number of edges from traversals of BRON given the records

In [5]:
%%time
_ = download_report_extract_ttps_query_bron(
    BRON_SERVER_IP, BRON_PASSWORD, BRON_USERNAME, url=THREAT_REPORT_URL
)

Query results records: {
 "tactic": {
  "TA0006": {
   "technique": 49
  }
 },
 "technique": {
  "T1552.004": {
   "tactic": 1
  },
  "T1114.002": {
   "tactic": 1
  },
  "T1552": {
   "tactic": 1
  },
  "T1114": {
   "tactic": 1
  },
  "T1199": {
   "tactic": 1
  }
 },
 "cve": {}
}
Query results number of traversals: {
 "tactic": 6,
 "technique": 9,
 "cve": 3
}
CPU times: user 9.84 s, sys: 1.04 s, total: 10.9 s
Wall time: 52.3 s


## Example using file with a network description

As above but the results are filtered based on CPE format matches with the affected product configurations listed in a network description file

In [6]:
%%time
network_description_file = "graph_db/example_data/network_file_bron.json"
_ = download_report_extract_ttps_query_bron(
    BRON_SERVER_IP,
    BRON_PASSWORD,
    BRON_USERNAME,
    url=THREAT_REPORT_URL,
    network_description_file=network_description_file,
)

Query results records: {
 "tactic": {
  "TA0006": {
   "technique": 49
  }
 },
 "technique": {
  "T1552.004": {
   "tactic": 1
  },
  "T1114.002": {
   "tactic": 1
  },
  "T1552": {
   "tactic": 1
  },
  "T1114": {
   "tactic": 1
  },
  "T1199": {
   "tactic": 1
  }
 },
 "cve": {}
}
Query results number of traversals: {
 "tactic": 6,
 "technique": 9,
 "cve": 3
}
Number of configurations in CPE format in network 18
Network matches: {
 "tactic": [
  "TA0006"
 ]
}
CPU times: user 9.93 s, sys: 1.06 s, total: 11 s
Wall time: 51.6 s
